<a href="https://colab.research.google.com/github/ksk94966/XNN_Math/blob/master/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
import os.path
import urllib.request
import gzip
import math
import numpy  as np
import matplotlib.pyplot as plt

# data
DATA_NUM_TRAIN         = 60000
DATA_NUM_TEST          = 10000
DATA_CHANNELS          = 1
DATA_ROWS              = 28
DATA_COLS              = 28
DATA_CLASSES           = 10
DATA_URL_TRAIN_DATA    = 'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz'
DATA_URL_TRAIN_LABELS  = 'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz'
DATA_URL_TEST_DATA     = 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
DATA_URL_TEST_LABELS   = 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
DATA_FILE_TRAIN_DATA   = 'train_data.gz'
DATA_FILE_TRAIN_LABELS = 'train_labels.gz'
DATA_FILE_TEST_DATA    = 'test_data.gz'
DATA_FILE_TEST_LABELS  = 'test_labels.gz'

# display
DISPLAY_ROWS   = 8
DISPLAY_COLS   = 4
DISPLAY_COL_IN = 10
DISPLAY_ROW_IN = 25
DISPLAY_NUM    = DISPLAY_ROWS*DISPLAY_COLS

# download
if (os.path.exists(DATA_FILE_TRAIN_DATA)   == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_DATA,   DATA_FILE_TRAIN_DATA)
if (os.path.exists(DATA_FILE_TRAIN_LABELS) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_LABELS, DATA_FILE_TRAIN_LABELS)
if (os.path.exists(DATA_FILE_TEST_DATA)    == False):
    urllib.request.urlretrieve(DATA_URL_TEST_DATA,    DATA_FILE_TEST_DATA)
if (os.path.exists(DATA_FILE_TEST_LABELS)  == False):
    urllib.request.urlretrieve(DATA_URL_TEST_LABELS,  DATA_FILE_TEST_LABELS)

# training data
# unzip the file, skip the header, read the rest into a buffer and format to NCHW
file_train_data   = gzip.open(DATA_FILE_TRAIN_DATA, 'r')
file_train_data.read(16)
buffer_train_data = file_train_data.read(DATA_NUM_TRAIN*DATA_ROWS*DATA_COLS)
train_data        = np.frombuffer(buffer_train_data, dtype=np.uint8).astype(np.float32)
train_data        = train_data.reshape(DATA_NUM_TRAIN, 1, DATA_ROWS, DATA_COLS)

# training labels
# unzip the file, skip the header, read the rest into a buffer and format to a vector
file_train_labels   = gzip.open(DATA_FILE_TRAIN_LABELS, 'r')
file_train_labels.read(8)
buffer_train_labels = file_train_labels.read(DATA_NUM_TRAIN)
train_labels        = np.frombuffer(buffer_train_labels, dtype=np.uint8).astype(np.int32)

# testing data
# unzip the file, skip the header, read the rest into a buffer and format to NCHW
file_test_data   = gzip.open(DATA_FILE_TEST_DATA, 'r')
file_test_data.read(16)
buffer_test_data = file_test_data.read(DATA_NUM_TEST*DATA_ROWS*DATA_COLS)
test_data        = np.frombuffer(buffer_test_data, dtype=np.uint8).astype(np.float32)
test_data        = test_data.reshape(DATA_NUM_TEST, 1, DATA_ROWS, DATA_COLS)

# testing labels
# unzip the file, skip the header, read the rest into a buffer and format to a vector
file_test_labels   = gzip.open(DATA_FILE_TEST_LABELS, 'r')
file_test_labels.read(8)
buffer_test_labels = file_test_labels.read(DATA_NUM_TEST)
test_labels        = np.frombuffer(buffer_test_labels, dtype=np.uint8).astype(np.int32)

In [215]:
#Printing dimensions
print(train_data.shape)   # (60000, 1, 28, 28)
print(train_labels.shape) # (60000,)
print(test_data.shape)    # (10000, 1, 28, 28)
print(test_labels.shape)  # (10000,)

(60000, 1, 28, 28)
(60000,)
(10000, 1, 28, 28)
(10000,)


In [221]:
lr = 0.5

#hidden Layer1
wh1 = np.random.rand(784,1000)/np.sqrt(784)
bh1 = np.random.rand(1,1000)

#hidden Layer2
wh2 = np.random.rand(1000,100)/np.sqrt(1000)
bh2 = np.random.rand(1,100)

#Output Layer
wo = np.random.rand(100,10)/np.sqrt(100)
bo = np.random.rand(1,10)



In [222]:
def multi1(v):
  return np.dot(v,wh1)

def multi2(v):
  return np.dot(v,wh2)

def multi3(v):
  return np.dot(v,wo)

def ReLU(x):
  return np.maximum(0,x)

def reluDerivative(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x /np.sum(e_x)

def getLabelMatrix(i):
  lab = np.zeros((1,10))
  lab[0][i] = 1
  return lab



In [234]:
#Forward Pass
for epoch in range(5000):
  a0 = train_data[epoch]/255.0                                                      #division by 255
  a0 = a0.reshape(1,784)                                                                 #Reshaping ---> Vectorization
  # print(bh1)

  #Passing the sample through first hidden layer
  zh1 = np.add(multi1(a0) ,bh1)                                                                    #Creating Zh1 which is addtion of weights and bias
  ah1 = ReLU(zh1)                                                                                 #Generated hiddenlayer1 Activations
  # print(ah1)

  #passing the h1 sample through second hiddden layer
  zh2 = np.add(multi2(ah1), bh2)                                                                   #Creating Zh2 which is addtion of weights and bias
  ah2 = ReLU(zh2)                                                                                #Generated hiddenlayer2 Activations 
  

  #passing the h2 sample through third hiddden layer
  zo = np.add(multi3(ah2) ,bo)                                                                        #Creating Zo which is addtion of weights and bias
  ao = softmax(zo)                                                                                #Generated output Activations
  #print(ao)
  
  #Here we are using entropy function as loss function

  #BackPropagation----------------------------------------------------------

  #phase 1              ---> for the output layer

  y = getLabelMatrix(train_labels[epoch])
  dcost_dzo = ao - y                                                                             #(1,10)
  dzo_dwo = ah2.transpose()                                                            #(1,100)   --> we have to transpose this one

  dcost_dwo = np.dot( dzo_dwo,dcost_dzo)                                 #derivate of cost w.r.t output weights      #(100,10)
  dcost_dbo = dcost_dzo                                                                   #derivative of cost w.r.t bias                      #(1,10)

  #phase 2          ---> for the hidden layer 2 
  dzo_dah2 = wo
  dcost_dah2 = np.dot(dcost_dzo , dzo_dah2.transpose())
  dah2_dzh2 = reluDerivative(zh2)
  dzh2_dwh2 = ah1
  dcost_dwh2 = np.dot(dzh2_dwh2.T,dah2_dzh2*dcost_dah2)
  dcost_dbh2 = dcost_dah2 * dah2_dzh2
  #print(dcost_dwh2)


  #phase 3   ---- > for hidden layer 1
  dzh2_dah1 = wh2
  dcost_dzh2 = dcost_dah2 * dah2_dzh2
  dcost_dah1 = np.dot(dcost_dzh2,dzh2_dah1.transpose()) 
  dah1_dzh1 = reluDerivative(zh1)
  dzh1_dwh1 = a0
  dcost_dwh1 = np.dot(dzh1_dwh1.T,dah1_dzh1 *dcost_dah1)
  dcost_dbh1 = dcost_dah1 * dah1_dzh1
  #print(dzh1_dwh1)

  wh1 -= lr * dcost_dwh1
  bh1 -= lr * dcost_dbh1.sum(axis=0)

  wh2 -= lr * dcost_dwh2
  bh2 -= lr * dcost_dbh2.sum(axis=0)

  wo -= lr * dcost_dwo
  bo -= lr * dcost_dbo.sum(axis=0)

  if epoch % 10 == 0:
        loss = np.sum(-y * np.log(ao))
        print('Loss function value: ', loss)

Loss function value:  2.4440654471655012
Loss function value:  2.1428071637286563
Loss function value:  2.727416201260738
Loss function value:  2.0845699011286096
Loss function value:  2.4162781719019444
Loss function value:  1.8436296251306923
Loss function value:  2.315761913421953
Loss function value:  2.341440733311132
Loss function value:  2.7659789095345335
Loss function value:  2.29788267741841
Loss function value:  3.375219706394253
Loss function value:  2.9505918167412117
Loss function value:  2.6500701885973053
Loss function value:  2.7424702572316817
Loss function value:  2.7432298513612974
Loss function value:  2.163145345564577
Loss function value:  2.522200385467925
Loss function value:  1.9593662364752662
Loss function value:  1.9687056793247348
Loss function value:  1.3405813619569567
Loss function value:  2.6937260478346294
Loss function value:  2.1754762379118278
Loss function value:  2.2774844380013723
Loss function value:  2.2081647898387096
Loss function value:  2.